In [ ]:
beginDate = '01/01/2000'
endDate = '01/01/2100'

In [ ]:
date_df = spark.sql("""select explode(sequence(to_date('2000-01-01'), to_date('2100-12-31'), interval 1 day)) as date_tmp""")
#display(date_df)

In [ ]:
def add_sufix(day: str):
    day_as_int = int(day)
    if day_as_int < 10:
        if day_as_int == 1:
            return "1st"
        if day_as_int == 2:
            return "2nd"
        if day_as_int == 3:
            return "3rd"
        else:
            return str(day_as_int) + "th"        
    elif day_as_int in [11,12,13]:
        return str(day_as_int) + "th"
    else:
        if day[1] == '1':
            return day[0] + "1st"
        if day[1] == '2':
            return day[0] + "2nd"
        if day[1] == '3':
            return day[0] + "3rd"
        else:
            return str(day_as_int) + "th"

def get_day_of_quarter(current_date: DateType()):

    quarter_days = [date(current_date.year, i, 1) for i in [1, 4, 7,10]]
    current_date = date(current_date.year, current_date.month, current_date.day)

    if current_date < quarter_days[1]:
        delta = current_date - quarter_days[0]
        return(delta.days + 1)
    elif  current_date < quarter_days[2]:
        delta = current_date - quarter_days[1]
        return(delta.days + 1)
    elif  current_date < quarter_days[3]:
        delta = current_date - quarter_days[2]
        return(delta.days + 1)
    else:
        delta = current_date - quarter_days[3]
        return(delta.days + 1)

from math import ceil

def get_week_of_month(dt: DateType()):
    """ Returns the week of the month for the specified date.
    """
    first_day = dt.replace(day=1)
    print(first_day)

    dom = dt.day
    print(dom)

    adjusted_dom = dom + first_day.weekday() + 1

    print(adjusted_dom)

    return int(ceil(adjusted_dom/7.0))

def get_week_of_quarter(current_date: DateType()):
    """ Returns the week of the month for the specified date.
    """
    quarter_days = [date(current_date.year, i, 1) for i in [1, 4, 7,10]]
    current_date = date(current_date.year, current_date.month, current_date.day)

    #defines when to start counting
    if current_date < quarter_days[1]:
        first_day = quarter_days[0]
    elif  current_date < quarter_days[2]:
        first_day = quarter_days[1]
    elif  current_date < quarter_days[3]:
        first_day = quarter_days[2]
    else:
        first_day = quarter_days[3]

    #number of days passed in the quarter
    dom = get_day_of_quarter(current_date)

    #adjust it to start couting from saturdays
    adjusted_dom = dom + first_day.weekday() + 1

    return int(ceil(adjusted_dom/7.0))

def get_week_of_year(current_date: DateType()):
    """ The weekofyear function of spark is wrong
    """
    current_date = date(current_date.year, current_date.month, current_date.day)
    first_day = date(current_date.year, 1, 1)
    print(first_day)

    date_diff = current_date - first_day
    dom = date_diff.days + 1
    print(dom)

    adjusted_dom = dom + first_day.weekday() + 1

    print(adjusted_dom)

    return int(ceil(adjusted_dom/7.0))

def get_quarter_name(quarter: str):
    if quarter == "1":
        return ("First")
    if quarter == "2":
        return ("Second")
    if quarter == "3":
        return ("Third")
    else:
        return ("Fourth")

def get_month_of_quarter(month: str):
    return int(month) - ceil(int(month)/4) + 1

import calendar 

def get_first_day(current_date: date):
        return date(current_date.year, current_date.month, 1)

def get_last_day(current_date: date):
    return date(current_date.year, current_date.month, calendar.monthrange(current_date.year, current_date.month)[1])

def is_weekday(day_no: int):
    if day_no == 7 or day_no == 1:
        return False
    else:
        return True
#a = get_first_day(datetime.now(), 2)
#print(a)

is_weekday_udf = udf(lambda x: is_weekday(x), BooleanType())

get_last_day_udf = udf(lambda x: get_last_day(x), DateType())

get_first_day_udf = udf(lambda x: get_first_day(x), DateType())

get_month_of_quarter_udf = udf(lambda x: get_month_of_quarter(x), IntegerType())

get_week_of_quarter_udf = udf(lambda x: get_week_of_quarter(x), StringType())

get_quarter_name_udf = udf(lambda x: get_quarter_name(x), StringType())

get_week_of_year_udf = udf(lambda x: get_week_of_year(x), IntegerType())

get_week_of_month_udf =  udf(lambda x: get_week_of_month(x), IntegerType())

get_day_of_quarter_udf = udf(lambda x: get_day_of_quarter(x), IntegerType())

get_day_of_quarter_udf = udf(lambda x: get_day_of_quarter(x), IntegerType())

add_sufix_udf = udf(lambda x: add_sufix(x), StringType())


In [ ]:
date_df2 = date_df\
    .withColumn("sk_date", date_format(col("date_tmp"), "yyyyMMdd").cast("int"))\
    .withColumn("data", to_timestamp("date_tmp", 'MM-dd-yyyy HH:mm:ss.SSSS'))\
    .withColumn("full_date", date_format("date_tmp", 'dd/MM/yyyy'))\
    .withColumn("day_of_month", date_format(col("date_tmp"), "dd"))\
    .withColumn("day_suffix", add_sufix_udf("day_of_month"))\
    .withColumn("day_name", date_format("date_tmp", "EEEE"))\
    .withColumn("day_of_week", dayofweek("date_tmp").cast("string"))\
    .withColumn("day_of_quarter", get_day_of_quarter_udf(col("date_tmp")).cast("string"))\
    .withColumn("day_of_year", dayofyear("date_tmp").cast("string"))\
    .withColumn("week_of_month", get_week_of_month_udf("date_tmp").cast("string"))\
    .withColumn("week_of_quarter", get_week_of_quarter_udf("date_tmp").cast("string"))\
    .withColumn("week_of_year", get_week_of_year_udf("date_tmp").cast("string"))\
    .withColumn("month", date_format(col("date_tmp"), "MM").cast("string"))\
    .withColumn("month_name", date_format(col("date_tmp"), "MMMM"))\
    .withColumn("month_of_quarter", get_month_of_quarter_udf(date_format("date_tmp", "M")).cast("string"))\
    .withColumn("quarter", date_format(col("date_tmp"), "Q"))\
    .withColumn("quarter_name", get_quarter_name_udf(date_format(col("date_tmp"), "Q")))\
    .withColumn("year", date_format(col("date_tmp"), "yyyy"))\
    .withColumn("month_year", date_format(col("date_tmp"), "MMM-yy"))\
    .withColumn("MMYYYY", date_format(col("date_tmp"), "MMyyyy"))\
    .withColumn("first_day_of_month", get_first_day_udf("date_tmp"))\
    .withColumn("last_day_of_month", get_last_day_udf("date_tmp"))\
    .withColumn("is_weekday", is_weekday_udf(dayofweek("date_tmp")))


date_df3 = date_df2.drop(col("date_tmp"))
    
display(date_df3)